In [1]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from collections import Counter
from fastFM import sgd
import lightgbm as lgb
import scipy.sparse as sparse
import pandas as pd
import numpy as np
import tqdm
import gc
import os
import sys
sys.path.append('../../../code/utils')
sys.path.append('../../../code/pipeline')
sys.path.append('../../../code')
import data_utils as du
import perf_utils as pu
import data_jointer as dj
import config

In [2]:
df_train = du.load_raw_data("train")
y = df_train['label'].values

In [3]:
input_folder = "../../../data/input"
input_file = "train.raw.binary.pkl"
input_path = os.path.join(input_folder, input_file)

with pu.profiler("loading train binary data"):
    (feat_names, X_tv) = du.load_pickle(input_path)

[07:48:57] Finish loading train binary data. △M: +3.26GB. △T: 5.4 seconds.


In [11]:
print("Average non-zero entry per row: {:.1f}".format(X_tv.getnnz() / X_tv.shape[0]))

Average non-zero entry per row: 78.2


In [17]:
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits)
split_indices = [(train_index, valid_index) for train_index, valid_index in skf.split(df_train, y)]

In [19]:
with pu.profiler("splitting training and validation set"):
    train_index, valid_index = split_indices[0]

    X_train, y_train = X_tv[train_index, :], y[train_index]
    X_valid, y_valid = X_tv[valid_index, :], y[valid_index]
    gc.collect()
#     X_train, y_train = X_tv[valid_index, :], y[valid_index]
#     X_valid, y_valid = X_tv[train_index, :], y[train_index]

[08:14:27] Finish splitting training and validation set. △M: -9.69GB. △T: 18.4 seconds.


In [69]:
def format_params(params, jointer= ", "):
    strings = []
    for k, v in params.items():
        if k == "n_iter":
            strings.append("n_iter: {}".format(round(v)))
        elif k == "init_stdev":
            strings.append("init_stdev: {:.6f}".format(v))
        elif k == "rank":
            strings.append("rank: {:>4}".format(v))
        elif k == "l2_reg_w":
            strings.append("l2_w: {:.3g}".format(v))
        elif k == "l2_reg_V":
            strings.append("l2_V: {:.6f}".format(v))
        elif k == "step_size":
            strings.append("lr: {:.3g}".format(v))
    return jointer.join(strings)

In [41]:
frozen_params = {
    'init_stdev': 0.01,
    'step_size': 0.05,
    'n_iter': 1000
}


def objective(params):
    for k, v in params.items():
        if k in set(["n_iter", "rank"]):
            params[k] = round(v)
    params.update(frozen_params)
    
    print("[{}] start trying params: {}".format(pu.get_time_str(), format_params(params)))
    fm = sgd.FMClassification(**params)

    with pu.profiler("fitting FM for {} more round(s)".format(n_iter)):
        fm.fit(X_train, y_train)
        
    with pu.profiler("making prediction on validation set"):
        proba_valid = fm.predict_proba(X_valid)
        del fm
        gc.collect()
        
    if np.isnan(proba_valid).sum() == 0:
        auc = metrics.roc_auc_score(y_valid, proba_valid)
        print("[{}] AUC: {:.6f}".format(pu.get_time_str(), auc))
        del proba_valid
        gc.collect()
        return {
            'params': params,
            'loss': -auc,
            'status': STATUS_OK
        }
    else:
        print("[{}] Failed".format(pu.get_time_str()))
        return {
            'params': params,
            'loss': 0,
            'status': STATUS_OK
        }

In [43]:
# frozen_params unchanged
trials = Trials()
best = fmin(objective,
            space={
                'rank': hp.loguniform('rank', np.log(5), np.log(50)),
                'l2_reg_w': hp.loguniform('l2_reg_w', np.log(1e-5), np.log(1e-1)),
                'l2_reg_V': hp.loguniform('l2_reg_V', np.log(1e-5), np.log(1))
            },
            algo=tpe.suggest,
            max_evals=100,
            trials=trials)

[08:51:36] start trying params: rank:   36, n_iter: 1000, l2_w: 0.000011, l2_V: 0.000232, init_stdev: 0.010000
[08:51:44] Finish fitting FM for 1 more round(s). △M: +115.28MB. △T: 7.9 seconds.
[08:52:19] Finish making prediction on validation set. △M: -1.61GB. △T: 34.7 seconds.
[08:52:19] AUC: 0.527958
[08:52:19] start trying params: rank:   48, n_iter: 1000, l2_w: 0.000253, l2_V: 0.241453, init_stdev: 0.010000
[08:52:27] Finish fitting FM for 1 more round(s). △M: +153.7MB. △T: 8.2 seconds.
[08:53:09] Finish making prediction on validation set. △M: -153.7MB. △T: 41.5 seconds.
[08:53:09] AUC: 0.564793
[08:53:09] start trying params: rank:    9, n_iter: 1000, l2_w: 0.038827, l2_V: 0.001605, init_stdev: 0.010000
[08:53:16] Finish fitting FM for 1 more round(s). △M: +0B. △T: 7.0 seconds.
[08:53:34] Finish making prediction on validation set. △M: +13.43MB. △T: 17.6 seconds.
[08:53:34] AUC: 0.553909
[08:53:34] start trying params: rank:   44, n_iter: 1000, l2_w: 0.009318, l2_V: 0.000046, ini

[09:05:20] Finish fitting FM for 1 more round(s). △M: +0B. △T: 7.0 seconds.
[09:05:45] Finish making prediction on validation set. △M: +80.43MB. △T: 25.0 seconds.
[09:05:45] AUC: 0.565273
[09:05:45] start trying params: rank:   11, n_iter: 1000, l2_w: 0.079524, l2_V: 0.000011, init_stdev: 0.010000
[09:05:52] Finish fitting FM for 1 more round(s). △M: +0B. △T: 7.1 seconds.
[09:06:12] Finish making prediction on validation set. △M: +53.58MB. △T: 19.3 seconds.
[09:06:12] AUC: 0.549355
[09:06:12] start trying params: rank:    9, n_iter: 1000, l2_w: 0.000118, l2_V: 0.077560, init_stdev: 0.010000
[09:06:19] Finish fitting FM for 1 more round(s). △M: +0B. △T: 7.2 seconds.
[09:06:38] Finish making prediction on validation set. △M: +53.58MB. △T: 18.7 seconds.
[09:06:39] AUC: 0.564359
[09:06:39] start trying params: rank:    6, n_iter: 1000, l2_w: 0.000012, l2_V: 0.314417, init_stdev: 0.010000
[09:06:45] Finish fitting FM for 1 more round(s). △M: +0B. △T: 6.6 seconds.
[09:07:01] Finish making pr

[09:18:31] AUC: 0.491199
[09:18:31] start trying params: rank:    8, n_iter: 1000, l2_w: 0.000093, l2_V: 0.001071, init_stdev: 0.010000
[09:18:38] Finish fitting FM for 1 more round(s). △M: +0B. △T: 6.8 seconds.
[09:18:54] Finish making prediction on validation set. △M: +0B. △T: 16.9 seconds.
[09:18:55] AUC: 0.560533
[09:18:55] start trying params: rank:    9, n_iter: 1000, l2_w: 0.000052, l2_V: 0.021282, init_stdev: 0.010000
[09:19:02] Finish fitting FM for 1 more round(s). △M: +0B. △T: 6.8 seconds.
[09:19:19] Finish making prediction on validation set. △M: +13.43MB. △T: 17.4 seconds.
[09:19:19] AUC: 0.562871
[09:19:19] start trying params: rank:    6, n_iter: 1000, l2_w: 0.018541, l2_V: 0.684773, init_stdev: 0.010000
[09:19:26] Finish fitting FM for 1 more round(s). △M: +0B. △T: 6.6 seconds.
[09:19:42] Finish making prediction on validation set. △M: +40.15MB. △T: 15.6 seconds.
[09:19:42] AUC: 0.562054
[09:19:42] start trying params: rank:    5, n_iter: 1000, l2_w: 0.000017, l2_V: 0.0

[09:30:40] Finish making prediction on validation set. △M: +40.15MB. △T: 17.3 seconds.
[09:30:41] AUC: 0.564510
[09:30:41] start trying params: rank:    5, n_iter: 1000, l2_w: 0.000454, l2_V: 0.954732, init_stdev: 0.010000
[09:30:47] Finish fitting FM for 1 more round(s). △M: +0B. △T: 6.6 seconds.
[09:31:02] Finish making prediction on validation set. △M: +0B. △T: 15.0 seconds.
[09:31:03] AUC: 0.565413
[09:31:03] start trying params: rank:    7, n_iter: 1000, l2_w: 0.000286, l2_V: 0.001478, init_stdev: 0.010000
[09:31:09] Finish fitting FM for 1 more round(s). △M: +0B. △T: 6.6 seconds.
[09:31:26] Finish making prediction on validation set. △M: +0B. △T: 16.4 seconds.
[09:31:26] AUC: 0.561178
[09:31:26] start trying params: rank:   30, n_iter: 1000, l2_w: 0.000082, l2_V: 0.451074, init_stdev: 0.010000
[09:31:34] Finish fitting FM for 1 more round(s). △M: +96.06MB. △T: 7.5 seconds.
[09:32:04] Finish making prediction on validation set. △M: -96.06MB. △T: 30.5 seconds.
[09:32:05] AUC: 0.565

In [44]:
print("Best Attempt:")
print(trials.best_trial['result'])

Best Attempt:
{'status': 'ok', 'params': {'rank': 5, 'step_size': 0.05, 'n_iter': 1000, 'l2_reg_w': 1.1449521114694572e-05, 'l2_reg_V': 0.9637349200060078, 'init_stdev': 0.01}, 'loss': -0.5655043634289212}


In [45]:
# frozen_params unchanged
trials2 = Trials()
best = fmin(objective,
            space={
                'rank': hp.loguniform('rank', np.log(50), np.log(100)),
                'l2_reg_w': hp.loguniform('l2_reg_w', np.log(1e-5), np.log(1e-1)),
                'l2_reg_V': hp.loguniform('l2_reg_V', np.log(1e-5), np.log(1))
            },
            algo=tpe.suggest,
            max_evals=100,
            trials=trials2)

[09:39:26] start trying params: rank:   88, n_iter: 1000, l2_w: 0.023981, l2_V: 0.039741, init_stdev: 0.010000
[09:39:36] Finish fitting FM for 1 more round(s). △M: +281.79MB. △T: 9.4 seconds.
[09:40:42] Finish making prediction on validation set. △M: -281.79MB. △T: 1.1 minutes.
[09:40:42] AUC: 0.521737
[09:40:42] start trying params: rank:   74, n_iter: 1000, l2_w: 0.000582, l2_V: 0.024757, init_stdev: 0.010000
[09:40:51] Finish fitting FM for 1 more round(s). △M: +236.95MB. △T: 8.7 seconds.
[09:41:48] Finish making prediction on validation set. △M: -236.95MB. △T: 57.2 seconds.
[09:41:48] AUC: 0.491792
[09:41:48] start trying params: rank:   76, n_iter: 1000, l2_w: 0.000012, l2_V: 0.099758, init_stdev: 0.010000
[09:41:57] Finish fitting FM for 1 more round(s). △M: +243.36MB. △T: 8.9 seconds.
[09:42:55] Finish making prediction on validation set. △M: -243.36MB. △T: 58.5 seconds.
[09:42:56] AUC: 0.562914
[09:42:56] start trying params: rank:   64, n_iter: 1000, l2_w: 0.046944, l2_V: 0.0

[10:07:57] AUC: 0.565081
[10:07:57] start trying params: rank:   67, n_iter: 1000, l2_w: 0.000073, l2_V: 0.007293, init_stdev: 0.010000
[10:08:06] Finish fitting FM for 1 more round(s). △M: +214.54MB. △T: 8.5 seconds.
[10:08:59] Finish making prediction on validation set. △M: -214.54MB. △T: 53.0 seconds.
[10:08:59] AUC: 0.487507
[10:08:59] start trying params: rank:   53, n_iter: 1000, l2_w: 0.000422, l2_V: 0.860294, init_stdev: 0.010000
[10:09:07] Finish fitting FM for 1 more round(s). △M: +169.71MB. △T: 8.0 seconds.
[10:09:52] Finish making prediction on validation set. △M: -169.71MB. △T: 44.4 seconds.
[10:09:52] AUC: 0.564974
[10:09:52] start trying params: rank:   58, n_iter: 1000, l2_w: 0.000273, l2_V: 0.122903, init_stdev: 0.010000
[10:10:00] Finish fitting FM for 1 more round(s). △M: +185.72MB. △T: 8.2 seconds.
[10:10:48] Finish making prediction on validation set. △M: -185.72MB. △T: 47.5 seconds.
[10:10:48] AUC: 0.565143
[10:10:48] start trying params: rank:   59, n_iter: 1000,

[10:35:44] AUC: 0.564937
[10:35:44] start trying params: rank:   55, n_iter: 1000, l2_w: 0.000117, l2_V: 0.032398, init_stdev: 0.010000
[10:35:53] Finish fitting FM for 1 more round(s). △M: +176.12MB. △T: 8.2 seconds.
[10:36:38] Finish making prediction on validation set. △M: -176.12MB. △T: 45.7 seconds.
[10:36:38] AUC: 0.484119
[10:36:39] start trying params: rank:   69, n_iter: 1000, l2_w: 0.000689, l2_V: 0.001924, init_stdev: 0.010000
[10:36:47] Finish fitting FM for 1 more round(s). △M: +220.95MB. △T: 8.6 seconds.
[10:37:42] Finish making prediction on validation set. △M: -220.95MB. △T: 54.9 seconds.
[10:37:42] AUC: 0.496225
[10:37:42] start trying params: rank:   51, n_iter: 1000, l2_w: 0.000015, l2_V: 0.004716, init_stdev: 0.010000
[10:37:50] Finish fitting FM for 1 more round(s). △M: +163.31MB. △T: 8.1 seconds.
[10:38:34] Finish making prediction on validation set. △M: -163.31MB. △T: 43.4 seconds.
[10:38:34] AUC: 0.502552
[10:38:34] start trying params: rank:   60, n_iter: 1000,

KeyboardInterrupt: 

In [46]:
print("Best Attempt:")
print(trials2.best_trial['result'])

Best Attempt:
{'status': 'ok', 'params': {'rank': 60, 'step_size': 0.05, 'n_iter': 1000, 'l2_reg_w': 0.00024384721330028536, 'l2_reg_V': 0.16812423237160723, 'init_stdev': 0.01}, 'loss': -0.5652250596240829}


In [47]:
trials3 = Trials()
best = fmin(objective,
            space={
                'rank': hp.loguniform('rank', np.log(50), np.log(100)),
                'l2_reg_w': hp.loguniform('l2_reg_w', np.log(1e-5), np.log(1e-1)),
                'l2_reg_V': hp.loguniform('l2_reg_V', np.log(1e-1), np.log(100))
            },
            algo=tpe.suggest,
            max_evals=30,
            trials=trials3)

[10:46:28] start trying params: rank:   72, n_iter: 1000, l2_w: 0.000023, l2_V: 4.475261, init_stdev: 0.010000
[10:46:37] Finish fitting FM for 1 more round(s). △M: +230.55MB. △T: 8.8 seconds.
[10:47:33] Finish making prediction on validation set. △M: -3.0GB. △T: 56.3 seconds.
[10:47:34] AUC: 0.565594
[10:47:34] start trying params: rank:   66, n_iter: 1000, l2_w: 0.001067, l2_V: 0.813930, init_stdev: 0.010000
[10:47:42] Finish fitting FM for 1 more round(s). △M: +211.34MB. △T: 8.5 seconds.
[10:48:35] Finish making prediction on validation set. △M: -211.34MB. △T: 52.4 seconds.
[10:48:35] AUC: 0.564692
[10:48:35] start trying params: rank:   77, n_iter: 1000, l2_w: 0.000694, l2_V: 5.089931, init_stdev: 0.010000
[10:48:44] Finish fitting FM for 1 more round(s). △M: +246.56MB. △T: 9.0 seconds.
[10:49:45] Finish making prediction on validation set. △M: -246.56MB. △T: 1.0 minutes.
[10:49:45] AUC: 0.565486
[10:49:45] start trying params: rank:   72, n_iter: 1000, l2_w: 0.000044, l2_V: 0.5220

KeyboardInterrupt: 

In [48]:
trials4 = Trials()
best = fmin(objective,
            space={
                'rank': hp.loguniform('rank', np.log(100), np.log(500)),
                'l2_reg_w': hp.loguniform('l2_reg_w', np.log(1e-5), np.log(1e-1)),
                'l2_reg_V': hp.loguniform('l2_reg_V', np.log(1), np.log(1000))
            },
            algo=tpe.suggest,
            max_evals=10,
            trials=trials4)

[10:58:06] start trying params: rank:  218, n_iter: 1000, l2_w: 0.005755, l2_V: 1.538308, init_stdev: 0.010000
[10:58:20] Finish fitting FM for 1 more round(s). △M: +698.05MB. △T: 13.8 seconds.
[11:00:48] Finish making prediction on validation set. △M: -2.42GB. △T: 2.5 minutes.
[11:00:48] AUC: 0.563954
[11:00:48] start trying params: rank:  178, n_iter: 1000, l2_w: 0.007808, l2_V: 658.831746, init_stdev: 0.010000
[11:01:00] Finish fitting FM for 1 more round(s). △M: +569.97MB. △T: 12.3 seconds.


/usr/local/lib/python3.5/dist-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/usr/local/lib/python3.5/dist-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/usr/local/lib/python3.5/dist-packages/scipy/stats/_distn_infrastructure.py:1738: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= self.b) & cond0


[11:03:02] Finish making prediction on validation set. △M: -569.77MB. △T: 2.0 minutes.
[11:03:02] Failed
[11:03:02] start trying params: rank:  134, n_iter: 1000, l2_w: 0.000040, l2_V: 2.981530, init_stdev: 0.010000
[11:03:13] Finish fitting FM for 1 more round(s). △M: +429.08MB. △T: 10.9 seconds.
[11:04:48] Finish making prediction on validation set. △M: -429.08MB. △T: 1.6 minutes.
[11:04:48] AUC: 0.565464
[11:04:48] start trying params: rank:  299, n_iter: 1000, l2_w: 0.021715, l2_V: 498.860807, init_stdev: 0.010000
[11:05:05] Finish fitting FM for 1 more round(s). △M: +957.42MB. △T: 16.4 seconds.
[11:08:21] Finish making prediction on validation set. △M: -957.42MB. △T: 3.3 minutes.
[11:08:21] Failed
[11:08:21] start trying params: rank:  324, n_iter: 1000, l2_w: 0.001411, l2_V: 4.666488, init_stdev: 0.010000
[11:08:39] Finish fitting FM for 1 more round(s). △M: +1.01GB. △T: 17.4 seconds.
[11:12:11] Finish making prediction on validation set. △M: -1.01GB. △T: 3.5 minutes.
[11:12:11] 

In [50]:
frozen_params = {
    'init_stdev': 0.01,
    'step_size': 0.05,
    # 'n_iter': 1000
}
trials5 = Trials()
best = fmin(objective,
            space={
                'rank': hp.loguniform('rank', np.log(50), np.log(100)),
                'l2_reg_w': hp.loguniform('l2_reg_w', np.log(1e-5), np.log(1e-1)),
                'l2_reg_V': hp.loguniform('l2_reg_V', np.log(1e-2), np.log(10)),
                'n_iter': hp.loguniform('n_iter', np.log(2000), np.log(10000))
            },
            algo=tpe.suggest,
            max_evals=30,
            trials=trials5)

[12:22:36] start trying params: rank:   83, n_iter: 3575, l2_w: 0.042700, l2_V: 0.049692, init_stdev: 0.010000
[12:22:51] Finish fitting FM for 1 more round(s). △M: +265.77MB. △T: 14.8 seconds.
[12:24:44] Finish making prediction on validation set. △M: -2.01GB. △T: 1.9 minutes.
[12:24:44] AUC: 0.496992
[12:24:44] start trying params: rank:   66, n_iter: 3979, l2_w: 0.002872, l2_V: 0.119636, init_stdev: 0.010000
[12:25:00] Finish fitting FM for 1 more round(s). △M: +211.34MB. △T: 15.6 seconds.
[12:26:34] Finish making prediction on validation set. △M: -211.34MB. △T: 1.6 minutes.
[12:26:34] AUC: 0.502465
[12:26:34] start trying params: rank:   81, n_iter: 6365, l2_w: 0.000222, l2_V: 0.270560, init_stdev: 0.010000
[12:26:46] Finish fitting FM for 1 more round(s). △M: +259.37MB. △T: 12.5 seconds.
[12:28:29] Finish making prediction on validation set. △M: -259.37MB. △T: 1.7 minutes.
[12:28:30] AUC: 0.518723
[12:28:30] start trying params: rank:   69, n_iter: 8108, l2_w: 0.016256, l2_V: 0.30

[13:08:39] AUC: 0.594238
[13:08:39] start trying params: rank:   53, n_iter: 9920, l2_w: 0.000025, l2_V: 0.504665, init_stdev: 0.010000
[13:08:54] Finish fitting FM for 1 more round(s). △M: +169.71MB. △T: 14.2 seconds.
[13:10:10] Finish making prediction on validation set. △M: -169.71MB. △T: 1.3 minutes.
[13:10:10] AUC: 0.508248
[13:10:10] start trying params: rank:   66, n_iter: 9183, l2_w: 0.092621, l2_V: 0.084570, init_stdev: 0.010000
[13:10:24] Finish fitting FM for 1 more round(s). △M: +211.34MB. △T: 13.7 seconds.
[13:11:53] Finish making prediction on validation set. △M: -211.34MB. △T: 1.5 minutes.
[13:11:54] AUC: 0.500922
[13:11:54] start trying params: rank:   59, n_iter: 6519, l2_w: 0.000119, l2_V: 1.385416, init_stdev: 0.010000
[13:12:08] Finish fitting FM for 1 more round(s). △M: +188.92MB. △T: 14.2 seconds.
[13:13:30] Finish making prediction on validation set. △M: -188.92MB. △T: 1.4 minutes.
[13:13:31] AUC: 0.599381


In [51]:
print("Best Attempt:")
print(trials5.best_trial['result'])

Best Attempt:
{'status': 'ok', 'params': {'rank': 50, 'step_size': 0.05, 'n_iter': 9997, 'l2_reg_w': 1.0412799773158667e-05, 'l2_reg_V': 0.9590030486516697, 'init_stdev': 0.01}, 'loss': -0.6000944928632376}


In [56]:
frozen_params = {
    'init_stdev': 0.01,
    'step_size': 0.05,
    # 'n_iter': 1000
}
trials6 = Trials()
best = fmin(objective,
            space={
                'rank': hp.loguniform('rank', np.log(50), np.log(200)),
                'l2_reg_w': hp.loguniform('l2_reg_w', np.log(1e-7), np.log(1e-4)),
                'l2_reg_V': hp.loguniform('l2_reg_V', np.log(5e-1), np.log(10)),
                'n_iter': hp.loguniform('n_iter', np.log(8000), np.log(30000))
            },
            algo=tpe.suggest,
            max_evals=30,
            trials=trials6)

[15:15:56] start trying params: rank:   55, n_iter: 22020, l2_w: 1.97e-07, l2_V: 1.561527, init_stdev: 0.010000
[15:16:06] Finish fitting FM for 1 more round(s). △M: +176.12MB. △T: 10.3 seconds.
[15:16:55] Finish making prediction on validation set. △M: -6.43GB. △T: 49.3 seconds.
[15:16:55] AUC: 0.485610
[15:16:55] start trying params: rank:   93, n_iter: 16104, l2_w: 6.01e-05, l2_V: 4.974036, init_stdev: 0.010000
[15:17:08] Finish fitting FM for 1 more round(s). △M: +297.79MB. △T: 12.5 seconds.


KeyboardInterrupt: 

In [59]:
frozen_params = {
    'init_stdev': 0.01,
    'step_size': 0.05,
    'l2_reg_V': 0.96,
    'rank': 50,
    'n_iter': 10000
}
trials6 = Trials()
best = fmin(objective,
            space={
                'l2_reg_w': hp.loguniform('l2_reg_w', np.log(1e-8), np.log(1e-4)),
                'n_iter': hp.uniform('n_iter', 6000, 12000)
            },
            algo=tpe.suggest,
            max_evals=30,
            trials=trials6)

[15:31:00] start trying params: rank:   50, n_iter: 10000, l2_w: 1.88e-07, l2_V: 0.960000, init_stdev: 0.010000
[15:31:10] Finish fitting FM for 1 more round(s). △M: +160.11MB. △T: 9.1 seconds.
[15:31:54] Finish making prediction on validation set. △M: -267.52MB. △T: 44.9 seconds.
[15:31:55] AUC: 0.599958
[15:31:55] start trying params: rank:   50, n_iter: 10000, l2_w: 8.86e-05, l2_V: 0.960000, init_stdev: 0.010000
[15:32:04] Finish fitting FM for 1 more round(s). △M: +160.11MB. △T: 9.0 seconds.
[15:32:49] Finish making prediction on validation set. △M: -160.11MB. △T: 45.5 seconds.
[15:32:50] AUC: 0.599949
[15:32:50] start trying params: rank:   50, n_iter: 10000, l2_w: 8.85e-06, l2_V: 0.960000, init_stdev: 0.010000
[15:32:59] Finish fitting FM for 1 more round(s). △M: +160.11MB. △T: 8.8 seconds.
[15:33:44] Finish making prediction on validation set. △M: -160.11MB. △T: 45.4 seconds.
[15:33:44] AUC: 0.599957
[15:33:44] start trying params: rank:   50, n_iter: 10000, l2_w: 5.19e-05, l2_V

KeyboardInterrupt: 

In [60]:
frozen_params = {
    'init_stdev': 0.01,
    'step_size': 0.05,
    'l2_reg_w': 1e-5,
    'rank': 50,
    'n_iter': 10000
}
trials7 = Trials()
best = fmin(objective,
            space={
                'l2_reg_V': hp.uniform('l2_reg_V', .9, 1.1)
            },
            algo=tpe.suggest,
            max_evals=30,
            trials=trials7)

[15:40:11] start trying params: rank:   50, n_iter: 10000, l2_w: 1e-05, l2_V: 0.956727, init_stdev: 0.010000
[15:40:20] Finish fitting FM for 1 more round(s). △M: +160.11MB. △T: 9.0 seconds.
[15:41:06] Finish making prediction on validation set. △M: -1.85GB. △T: 46.2 seconds.
[15:41:07] AUC: 0.599953
[15:41:07] start trying params: rank:   50, n_iter: 10000, l2_w: 1e-05, l2_V: 0.958412, init_stdev: 0.010000
[15:41:16] Finish fitting FM for 1 more round(s). △M: +160.11MB. △T: 9.0 seconds.
[15:42:00] Finish making prediction on validation set. △M: -160.11MB. △T: 44.4 seconds.
[15:42:01] AUC: 0.599955
[15:42:01] start trying params: rank:   50, n_iter: 10000, l2_w: 1e-05, l2_V: 0.955429, init_stdev: 0.010000
[15:42:10] Finish fitting FM for 1 more round(s). △M: +160.11MB. △T: 9.3 seconds.
[15:42:56] Finish making prediction on validation set. △M: -160.11MB. △T: 46.1 seconds.
[15:42:57] AUC: 0.599951
[15:42:57] start trying params: rank:   50, n_iter: 10000, l2_w: 1e-05, l2_V: 1.057021, in

[16:05:13] Finish fitting FM for 1 more round(s). △M: +160.11MB. △T: 9.0 seconds.
[16:05:59] Finish making prediction on validation set. △M: -160.11MB. △T: 46.1 seconds.
[16:05:59] AUC: 0.600020
[16:05:59] start trying params: rank:   50, n_iter: 10000, l2_w: 1e-05, l2_V: 1.096856, init_stdev: 0.010000
[16:06:10] Finish fitting FM for 1 more round(s). △M: +160.11MB. △T: 10.1 seconds.
[16:06:56] Finish making prediction on validation set. △M: -160.11MB. △T: 46.4 seconds.
[16:06:56] AUC: 0.600074
[16:06:56] start trying params: rank:   50, n_iter: 10000, l2_w: 1e-05, l2_V: 1.068859, init_stdev: 0.010000
[16:07:06] Finish fitting FM for 1 more round(s). △M: +160.11MB. △T: 9.2 seconds.
[16:07:54] Finish making prediction on validation set. △M: -160.11MB. △T: 48.7 seconds.
[16:07:55] AUC: 0.600058


In [61]:
print("Best Attempt:")
print(trials7.best_trial['result'])

Best Attempt:
{'status': 'ok', 'params': {'rank': 50, 'step_size': 0.05, 'n_iter': 10000, 'l2_reg_w': 1e-05, 'l2_reg_V': 1.0994499976284453, 'init_stdev': 0.01}, 'loss': -0.6000754489994073}


In [62]:
frozen_params = {
    'step_size': 0.05,
    'l2_reg_w': 1e-5,
    'l2_reg_V': 1.1,
    'rank': 50,
    'n_iter': 10000
}
trials7 = Trials()
best = fmin(objective,
            space={
                'init_stdev': hp.loguniform('init_stdev', np.log(1e-3), 0)
            },
            algo=tpe.suggest,
            max_evals=30,
            trials=trials7)

[16:09:51] start trying params: rank:   50, n_iter: 10000, l2_w: 1e-05, l2_V: 1.100000, init_stdev: 0.002068
[16:10:00] Finish fitting FM for 1 more round(s). △M: +160.11MB. △T: 9.2 seconds.
[16:10:46] Finish making prediction on validation set. △M: -160.11MB. △T: 46.1 seconds.
[16:10:47] AUC: 0.600739
[16:10:47] start trying params: rank:   50, n_iter: 10000, l2_w: 1e-05, l2_V: 1.100000, init_stdev: 0.001026
[16:10:56] Finish fitting FM for 1 more round(s). △M: +160.11MB. △T: 9.2 seconds.
[16:11:45] Finish making prediction on validation set. △M: -160.11MB. △T: 49.3 seconds.
[16:11:46] AUC: 0.600764
[16:11:46] start trying params: rank:   50, n_iter: 10000, l2_w: 1e-05, l2_V: 1.100000, init_stdev: 0.101305
[16:11:55] Finish fitting FM for 1 more round(s). △M: +160.11MB. △T: 9.3 seconds.
[16:12:44] Finish making prediction on validation set. △M: -160.11MB. △T: 48.8 seconds.
[16:12:44] AUC: 0.500265
[16:12:44] start trying params: rank:   50, n_iter: 10000, l2_w: 1e-05, l2_V: 1.100000, 

[16:34:57] Finish fitting FM for 1 more round(s). △M: +160.11MB. △T: 8.7 seconds.
[16:35:40] Finish making prediction on validation set. △M: -160.11MB. △T: 42.7 seconds.
[16:35:41] AUC: 0.600704
[16:35:41] start trying params: rank:   50, n_iter: 10000, l2_w: 1e-05, l2_V: 1.100000, init_stdev: 0.890951
[16:35:49] Finish fitting FM for 1 more round(s). △M: +160.11MB. △T: 8.7 seconds.
[16:36:32] Finish making prediction on validation set. △M: -160.11MB. △T: 42.7 seconds.
[16:36:32] AUC: 0.496464
[16:36:32] start trying params: rank:   50, n_iter: 10000, l2_w: 1e-05, l2_V: 1.100000, init_stdev: 0.001038
[16:36:41] Finish fitting FM for 1 more round(s). △M: +160.11MB. △T: 8.7 seconds.
[16:37:24] Finish making prediction on validation set. △M: -160.11MB. △T: 42.9 seconds.
[16:37:24] AUC: 0.600764


In [63]:
print("Best Attempt:")
print(trials7.best_trial['result'])

Best Attempt:
{'status': 'ok', 'params': {'rank': 50, 'step_size': 0.05, 'n_iter': 10000, 'l2_reg_w': 1e-05, 'l2_reg_V': 1.1, 'init_stdev': 0.0010264147152802211}, 'loss': -0.6007640704713189}


In [64]:
frozen_params = {
    'step_size': 0.05,
    'init_stdev': 0.001,
    'l2_reg_w': 1e-5,
    'rank': 25
}
trials8 = Trials()
best = fmin(objective,
            space={
                'l2_reg_V': hp.uniform('l2_reg_V', 0.4, 0.7),
                'n_iter': hp.uniform('n_iter', 6000, 12000)
            },
            algo=tpe.suggest,
            max_evals=30,
            trials=trials8)

[16:43:18] start trying params: rank:   25, n_iter: 8845, l2_w: 1e-05, l2_V: 0.617148, init_stdev: 0.001000
[16:43:25] Finish fitting FM for 1 more round(s). △M: +80.05MB. △T: 7.7 seconds.
[16:43:54] Finish making prediction on validation set. △M: -80.05MB. △T: 28.9 seconds.
[16:43:55] AUC: 0.572290
[16:43:55] start trying params: rank:   25, n_iter: 10898, l2_w: 1e-05, l2_V: 0.600656, init_stdev: 0.001000
[16:44:03] Finish fitting FM for 1 more round(s). △M: +80.05MB. △T: 8.0 seconds.
[16:44:34] Finish making prediction on validation set. △M: -80.05MB. △T: 31.0 seconds.
[16:44:34] AUC: 0.589022
[16:44:34] start trying params: rank:   25, n_iter: 9949, l2_w: 1e-05, l2_V: 0.430614, init_stdev: 0.001000
[16:44:42] Finish fitting FM for 1 more round(s). △M: +80.05MB. △T: 8.0 seconds.
[16:45:11] Finish making prediction on validation set. △M: -80.05MB. △T: 29.3 seconds.
[16:45:12] AUC: 0.595201
[16:45:12] start trying params: rank:   25, n_iter: 9307, l2_w: 1e-05, l2_V: 0.641160, init_stde

[17:00:18] Finish fitting FM for 1 more round(s). △M: +80.05MB. △T: 7.8 seconds.
[17:00:49] Finish making prediction on validation set. △M: -80.05MB. △T: 30.5 seconds.
[17:00:49] AUC: 0.543161
[17:00:49] start trying params: rank:   25, n_iter: 10636, l2_w: 1e-05, l2_V: 0.599171, init_stdev: 0.001000
[17:00:57] Finish fitting FM for 1 more round(s). △M: +80.05MB. △T: 8.1 seconds.
[17:01:29] Finish making prediction on validation set. △M: -80.05MB. △T: 32.3 seconds.
[17:01:30] AUC: 0.600487
[17:01:30] start trying params: rank:   25, n_iter: 8696, l2_w: 1e-05, l2_V: 0.540719, init_stdev: 0.001000
[17:01:38] Finish fitting FM for 1 more round(s). △M: +80.05MB. △T: 8.0 seconds.
[17:02:08] Finish making prediction on validation set. △M: -80.05MB. △T: 29.9 seconds.
[17:02:08] AUC: 0.572687


In [65]:
print("Best Attempt:")
print(trials8.best_trial['result'])

Best Attempt:
{'status': 'ok', 'params': {'rank': 25, 'step_size': 0.05, 'n_iter': 10148, 'l2_reg_w': 1e-05, 'l2_reg_V': 0.5746245506919946, 'init_stdev': 0.001}, 'loss': -0.6059081644016701}


In [70]:
frozen_params = {
    'init_stdev': 0.001,
    'l2_reg_w': 1e-5,
    'l2_reg_V': .57,
    'rank': 25
}
trials9 = Trials()
best = fmin(objective,
            space={
                'step_size': hp.loguniform('step_size', np.log(1e-2), np.log(1e-1)),
                'n_iter': hp.loguniform('n_iter', np.log(1500), np.log(25000))
            },
            algo=tpe.suggest,
            max_evals=50,
            trials=trials9)

[17:11:20] start trying params: rank:   25, lr: 0.0803, n_iter: 22733, l2_w: 1e-05, l2_V: 0.570000, init_stdev: 0.001000
[17:11:29] Finish fitting FM for 1 more round(s). △M: +80.05MB. △T: 8.3 seconds.


/usr/local/lib/python3.5/dist-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/usr/local/lib/python3.5/dist-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/usr/local/lib/python3.5/dist-packages/scipy/stats/_distn_infrastructure.py:1738: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= self.b) & cond0


[17:11:58] Finish making prediction on validation set. △M: -1.69GB. △T: 29.5 seconds.
[17:11:58] Failed
[17:11:58] start trying params: rank:   25, lr: 0.0278, n_iter: 11776, l2_w: 1e-05, l2_V: 0.570000, init_stdev: 0.001000
[17:12:06] Finish fitting FM for 1 more round(s). △M: +80.05MB. △T: 8.0 seconds.
[17:12:35] Finish making prediction on validation set. △M: -80.05MB. △T: 29.2 seconds.
[17:12:36] AUC: 0.591227
[17:12:36] start trying params: rank:   25, lr: 0.0605, n_iter: 10058, l2_w: 1e-05, l2_V: 0.570000, init_stdev: 0.001000
[17:12:44] Finish fitting FM for 1 more round(s). △M: +80.05MB. △T: 7.8 seconds.
[17:13:13] Finish making prediction on validation set. △M: -80.05MB. △T: 29.5 seconds.
[17:13:14] AUC: 0.605681
[17:13:14] start trying params: rank:   25, lr: 0.0821, n_iter: 1883, l2_w: 1e-05, l2_V: 0.570000, init_stdev: 0.001000
[17:13:21] Finish fitting FM for 1 more round(s). △M: +80.05MB. △T: 7.6 seconds.
[17:13:50] Finish making prediction on validation set. △M: -80.05MB

[17:28:24] AUC: 0.621110
[17:28:24] start trying params: rank:   25, lr: 0.0149, n_iter: 13381, l2_w: 1e-05, l2_V: 0.570000, init_stdev: 0.001000
[17:28:32] Finish fitting FM for 1 more round(s). △M: +80.05MB. △T: 8.0 seconds.
[17:29:01] Finish making prediction on validation set. △M: -80.05MB. △T: 29.0 seconds.
[17:29:01] AUC: 0.620474
[17:29:01] start trying params: rank:   25, lr: 0.0103, n_iter: 21054, l2_w: 1e-05, l2_V: 0.570000, init_stdev: 0.001000
[17:29:10] Finish fitting FM for 1 more round(s). △M: +80.05MB. △T: 8.2 seconds.
[17:29:40] Finish making prediction on validation set. △M: -80.05MB. △T: 30.4 seconds.
[17:29:40] AUC: 0.628094
[17:29:40] start trying params: rank:   25, lr: 0.0106, n_iter: 21911, l2_w: 1e-05, l2_V: 0.570000, init_stdev: 0.001000
[17:29:49] Finish fitting FM for 1 more round(s). △M: +80.05MB. △T: 8.3 seconds.
[17:30:19] Finish making prediction on validation set. △M: -80.05MB. △T: 30.7 seconds.
[17:30:20] AUC: 0.624507
[17:30:20] start trying params: r

In [71]:
print("Best Attempt:")
print(trials9.best_trial['result'])

Best Attempt:
{'status': 'ok', 'params': {'rank': 25, 'step_size': 0.010004506912715214, 'n_iter': 20897, 'l2_reg_w': 1e-05, 'l2_reg_V': 0.57, 'init_stdev': 0.001}, 'loss': -0.6302206273372501}


In [72]:
frozen_params = {
    'init_stdev': 0.001,
    'l2_reg_w': 1e-5,
    'l2_reg_V': .57,
    'rank': 25,
    'step_size': 0.01
}
trials10 = Trials()
best = fmin(objective,
            space={
                'n_iter': hp.loguniform('n_iter', np.log(5000), np.log(50000))
            },
            algo=tpe.suggest,
            max_evals=20,
            trials=trials10)

[02:25:51] start trying params: rank:   25, lr: 0.01, n_iter: 49349, l2_w: 1e-05, l2_V: 0.570000, init_stdev: 0.001000
[02:26:01] Finish fitting FM for 1 more round(s). △M: +80.51MB. △T: 9.9 seconds.
[02:26:32] Finish making prediction on validation set. △M: -79.48MB. △T: 30.4 seconds.
[02:26:32] AUC: 0.640535
[02:26:32] start trying params: rank:   25, lr: 0.01, n_iter: 9183, l2_w: 1e-05, l2_V: 0.570000, init_stdev: 0.001000
[02:26:40] Finish fitting FM for 1 more round(s). △M: +80.05MB. △T: 8.0 seconds.
[02:27:09] Finish making prediction on validation set. △M: -80.05MB. △T: 28.4 seconds.
[02:27:09] AUC: 0.608325
[02:27:09] start trying params: rank:   25, lr: 0.01, n_iter: 19227, l2_w: 1e-05, l2_V: 0.570000, init_stdev: 0.001000
[02:27:17] Finish fitting FM for 1 more round(s). △M: +80.05MB. △T: 8.2 seconds.
[02:27:46] Finish making prediction on validation set. △M: -80.05MB. △T: 28.4 seconds.
[02:27:46] AUC: 0.635126
[02:27:46] start trying params: rank:   25, lr: 0.01, n_iter: 183

In [73]:
print("Best Attempt:")
print(trials10.best_trial['result'])

Best Attempt:
{'status': 'ok', 'params': {'rank': 25, 'step_size': 0.01, 'n_iter': 49349, 'l2_reg_w': 1e-05, 'l2_reg_V': 0.57, 'init_stdev': 0.001}, 'loss': -0.6405346780576896}


In [74]:
frozen_params = {
    'init_stdev': 0.001,
    'l2_reg_w': 1e-5,
    'l2_reg_V': .57,
    'rank': 25,
    'step_size': 0.01
}
trials10 = Trials()
best = fmin(objective,
            space={
                'n_iter': hp.loguniform('n_iter', np.log(5000), np.log(5e4))
            },
            algo=tpe.suggest,
            max_evals=20,
            trials=trials10)

[02:59:52] start trying params: rank:   25, lr: 0.01, n_iter: 57076, l2_w: 1e-05, l2_V: 0.570000, init_stdev: 0.001000
[03:00:02] Finish fitting FM for 1 more round(s). △M: +80.05MB. △T: 10.1 seconds.
[03:00:31] Finish making prediction on validation set. △M: -80.05MB. △T: 28.8 seconds.
[03:00:31] AUC: 0.644565
[03:00:31] start trying params: rank:   25, lr: 0.01, n_iter: 57898, l2_w: 1e-05, l2_V: 0.570000, init_stdev: 0.001000
[03:00:41] Finish fitting FM for 1 more round(s). △M: +80.05MB. △T: 9.6 seconds.
[03:01:09] Finish making prediction on validation set. △M: -80.05MB. △T: 28.3 seconds.
[03:01:09] AUC: 0.642098
[03:01:09] start trying params: rank:   25, lr: 0.01, n_iter: 31141, l2_w: 1e-05, l2_V: 0.570000, init_stdev: 0.001000
[03:01:18] Finish fitting FM for 1 more round(s). △M: +80.05MB. △T: 8.9 seconds.
[03:01:47] Finish making prediction on validation set. △M: -80.05MB. △T: 28.9 seconds.
[03:01:48] AUC: 0.630466
[03:01:48] start trying params: rank:   25, lr: 0.01, n_iter: 7

In [75]:
print("Best Attempt:")
print(trials10.best_trial['result'])

Best Attempt:
{'status': 'ok', 'params': {'rank': 25, 'step_size': 0.01, 'n_iter': 57076, 'l2_reg_w': 1e-05, 'l2_reg_V': 0.57, 'init_stdev': 0.001}, 'loss': -0.6445645822673517}
The history saving thread hit an unexpected error (OperationalError('disk I/O error',)).History will not be written to the database.


In [76]:
frozen_params = {
    'init_stdev': 0.001,
    'l2_reg_w': 1e-5,
    'l2_reg_V': .57,
    'rank': 25
}
trials10 = Trials()
best = fmin(objective,
            space={
                'step_size': hp.uniform('step_size', 1e-3, 1e-2),
                'n_iter': hp.uniform('n_iter', 5e4, 1e6)
            },
            algo=tpe.suggest,
            max_evals=50,
            trials=trials10)

[04:06:12] start trying params: rank:   25, lr: 0.00189, n_iter: 680147, l2_w: 1e-05, l2_V: 0.570000, init_stdev: 0.001000
[04:06:45] Finish fitting FM for 1 more round(s). △M: +80.05MB. △T: 33.5 seconds.
[04:07:16] Finish making prediction on validation set. △M: -80.05MB. △T: 30.8 seconds.
[04:07:17] AUC: 0.670219
[04:07:17] start trying params: rank:   25, lr: 0.00656, n_iter: 269604, l2_w: 1e-05, l2_V: 0.570000, init_stdev: 0.001000
[04:07:34] Finish fitting FM for 1 more round(s). △M: +80.05MB. △T: 17.8 seconds.
[04:08:04] Finish making prediction on validation set. △M: -80.05MB. △T: 29.9 seconds.
[04:08:05] AUC: 0.659783
[04:08:05] start trying params: rank:   25, lr: 0.00781, n_iter: 462325, l2_w: 1e-05, l2_V: 0.570000, init_stdev: 0.001000
[04:08:30] Finish fitting FM for 1 more round(s). △M: +80.05MB. △T: 25.4 seconds.
[04:09:00] Finish making prediction on validation set. △M: -80.05MB. △T: 30.2 seconds.
[04:09:01] AUC: 0.663334
[04:09:01] start trying params: rank:   25, lr: 0

[04:32:01] AUC: 0.675915
[04:32:01] start trying params: rank:   25, lr: 0.00327, n_iter: 818702, l2_w: 1e-05, l2_V: 0.570000, init_stdev: 0.001000
[04:32:40] Finish fitting FM for 1 more round(s). △M: +80.05MB. △T: 39.1 seconds.
[04:33:11] Finish making prediction on validation set. △M: -80.05MB. △T: 31.4 seconds.
[04:33:12] AUC: 0.675231
[04:33:12] start trying params: rank:   25, lr: 0.0045, n_iter: 927356, l2_w: 1e-05, l2_V: 0.570000, init_stdev: 0.001000
[04:33:54] Finish fitting FM for 1 more round(s). △M: +80.05MB. △T: 42.4 seconds.
[04:34:24] Finish making prediction on validation set. △M: -80.05MB. △T: 29.7 seconds.
[04:34:24] AUC: 0.672071
[04:34:24] start trying params: rank:   25, lr: 0.00118, n_iter: 789322, l2_w: 1e-05, l2_V: 0.570000, init_stdev: 0.001000
[04:35:01] Finish fitting FM for 1 more round(s). △M: +80.05MB. △T: 37.1 seconds.
[04:35:32] Finish making prediction on validation set. △M: -80.05MB. △T: 31.0 seconds.
[04:35:33] AUC: 0.675158
[04:35:33] start trying p

In [77]:
print("Best Attempt:")
print(trials10.best_trial['result'])

Best Attempt:
{'status': 'ok', 'params': {'rank': 25, 'step_size': 0.003028193825915932, 'n_iter': 997351, 'l2_reg_w': 1e-05, 'l2_reg_V': 0.57, 'init_stdev': 0.001}, 'loss': -0.6782609116938664}


In [78]:
frozen_params = {
    'init_stdev': 0.001,
    'l2_reg_w': 1e-5,
    'l2_reg_V': .57,
    'rank': 25
}
trials10 = Trials()
best = fmin(objective,
            space={
                'step_size': hp.uniform('step_size', 5e-4, 3e-3),
                'n_iter': hp.uniform('n_iter', 1e6, 1e7)
            },
            algo=tpe.suggest,
            max_evals=50,
            trials=trials10)

[05:05:50] start trying params: rank:   25, lr: 0.000633, n_iter: 7230094, l2_w: 1e-05, l2_V: 0.570000, init_stdev: 0.001000
[05:10:30] Finish fitting FM for 1 more round(s). △M: +80.05MB. △T: 4.7 minutes.
[05:11:00] Finish making prediction on validation set. △M: -80.05MB. △T: 29.6 seconds.
[05:11:00] AUC: 0.690525
[05:11:00] start trying params: rank:   25, lr: 0.00127, n_iter: 1664317, l2_w: 1e-05, l2_V: 0.570000, init_stdev: 0.001000
[05:12:11] Finish fitting FM for 1 more round(s). △M: +80.05MB. △T: 1.2 minutes.
[05:12:41] Finish making prediction on validation set. △M: -80.05MB. △T: 29.2 seconds.
[05:12:41] AUC: 0.681780
[05:12:41] start trying params: rank:   25, lr: 0.00203, n_iter: 1680985, l2_w: 1e-05, l2_V: 0.570000, init_stdev: 0.001000
[05:13:53] Finish fitting FM for 1 more round(s). △M: +80.05MB. △T: 1.2 minutes.
[05:14:24] Finish making prediction on validation set. △M: -80.05MB. △T: 30.9 seconds.
[05:14:24] AUC: 0.681401
[05:14:24] start trying params: rank:   25, lr: 

[07:03:16] AUC: 0.690301
[07:03:16] start trying params: rank:   25, lr: 0.00121, n_iter: 9727148, l2_w: 1e-05, l2_V: 0.570000, init_stdev: 0.001000
[07:09:32] Finish fitting FM for 1 more round(s). △M: +80.05MB. △T: 6.3 minutes.
[07:10:04] Finish making prediction on validation set. △M: -80.05MB. △T: 31.8 seconds.
[07:10:04] AUC: 0.689457
[07:10:04] start trying params: rank:   25, lr: 0.000912, n_iter: 4613990, l2_w: 1e-05, l2_V: 0.570000, init_stdev: 0.001000
[07:13:08] Finish fitting FM for 1 more round(s). △M: +80.05MB. △T: 3.1 minutes.
[07:13:39] Finish making prediction on validation set. △M: -80.05MB. △T: 31.6 seconds.
[07:13:40] AUC: 0.688907
[07:13:40] start trying params: rank:   25, lr: 0.000525, n_iter: 7321546, l2_w: 1e-05, l2_V: 0.570000, init_stdev: 0.001000
[07:18:26] Finish fitting FM for 1 more round(s). △M: +80.05MB. △T: 4.8 minutes.
[07:18:57] Finish making prediction on validation set. △M: -80.05MB. △T: 30.7 seconds.
[07:18:57] AUC: 0.688180
[07:18:57] start tryin

In [79]:
print("Best Attempt:")
print(trials10.best_trial['result'])

Best Attempt:
{'status': 'ok', 'params': {'rank': 25, 'step_size': 0.0008691678873192621, 'n_iter': 8463250, 'l2_reg_w': 1e-05, 'l2_reg_V': 0.57, 'init_stdev': 0.001}, 'loss': -0.6925551857297931}


In [96]:
rows = []
trial_list = [trials, trials2, trials3, trials4, trials5, 
              trials6, trials7, trials8, trials9, trials10]
for t in trial_list:
    for result in t.results:
        try:
            row = {'auc': -result['loss']}
            for k, v in result['params'].items():
                row[k] = round(v, 7) if type(v) == float else v
            rows.append(row)
        except KeyError:
            pass
    df_log = pd.DataFrame(rows)
    df_log = df_log.sort_values("auc", ascending=False)

In [99]:
log_folder = "../../../log/fm/params_tunning"
log_file = "0505.csv"
log_path = os.path.join(log_folder, log_file)
os.makedirs(log_folder, exist_ok=True)

In [100]:
df_log.to_csv(log_path, index=False)